In [ ]:
import pandas as pd
import matplotlib.pyplot as pl
import numpy as np
import scipy.linalg
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('../data/zec_esm-cliff_ecs_tcr_co2.csv', index_col=0)

In [ ]:
df

In [ ]:
df.tcr/df.ecs

In [ ]:
# Data from Chris Jones, email 31.5.2023
tcr_ecs_all = np.array([0.487179487,0.49122807,0.395833333,0.583333333,0.555555556,0.642857143,0.5,0.615384615,0.666666667,0.681818182,0.535714286,0.620689655,0.705882353,0.5              ,0.514285714])
zec50_all   = np.array([0.01,-0.13,0.06,-0.27,-0.08,-0.27,0.28,0.01,-0.07,0.29,-0.04,0.01,-0.06,-0.36,0.03])
csink_all   = np.array([-0.45,-0.75,-0.8,-0.8,-0.75,-0.9,-0.6,-0.75,-0.75,-0.85,-1,-0.9,-0.7,-0.85,-0.6])

chrisj_models = [
    'access',
    'canesm',
    'cnrm',
    'gfdl',
    'miroc-es2l',
    'mpi',
    'ukesm',
    'bern',
    'dcess',
    'iapras',
    'loveclim',
    'mesm',
    'miroc-lite',
    'pgenie',
    'uvic',
]

esm_data_df = pd.DataFrame(
    {
        'tcr_ecs': tcr_ecs_all,
        'zec50': zec50_all,
        'csink': csink_all,
    },
    index = chrisj_models
)
esm_data_df

In [ ]:
# regular grid covering the domain of the data
X,Y = np.meshgrid(
    np.linspace(np.min(df.tcr/df.ecs), np.max(df.tcr/df.ecs)), 
    np.linspace(np.min(df.f50_1000), np.max(df.f50_1000))
)
XX = X.flatten()
YY = Y.flatten()

A = np.c_[df.tcr/df.ecs, df.f50_1000, np.ones_like(df.tcr/df.ecs)]
C,_,_,_ = scipy.linalg.lstsq(A, df.zec50_1000)    # coefficients

# evaluate it on grid
Z = C[0]*X + C[1]*Y + C[2]

In [ ]:
# check https://stackoverflow.com/questions/57367352/how-to-fit-a-plane-to-a-3d-dataset-in-python for coefficients

In [ ]:
fig = pl.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(df.tcr/df.ecs, df.f50_1000, df.zec50_1000)
ax.scatter(esm_data_df.tcr_ecs, esm_data_df.csink, esm_data_df.zec50)
ax.set_xlabel('TCR:ECS ratio')
ax.set_ylabel('delta CO$_2$ forcing')
ax.set_zlabel('ZEC50')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)

In [ ]:
z_pred= C[0]*df.tcr/df.ecs + C[1]*df.f50_1000 + C[2]
r2_score(df.zec50_1000, z_pred)

In [ ]:
def Adj_r2(Rsquared,n,p):
#n = number of data points, p = independent variables
    output = 1-(1-Rsquared)*(n-1)/(n-p-1)
    return output

Adj_r2(r2_score(df.zec50_1000, z_pred),1001,3)

In [ ]:
Z

In [ ]:
pl.scatter(df.tcr/df.ecs, df.zec50_1000)
pl.scatter(esm_data_df.tcr_ecs, esm_data_df.zec50)
pl.xlabel('TCR:ECS ratio')
pl.ylabel('ZEC50')

In [ ]:
pl.scatter(df.f50_1000, df.zec50_1000)
pl.scatter(esm_data_df.csink, esm_data_df.zec50)
pl.xlabel('Change in CO$_2$ radiative forcing, 50 years after net zero')
pl.ylabel('ZEC50')